In [0]:
include __DIR__ . '/vendor/autoload.php';

use Rubix\ML\Datasets\Labeled;
use Rubix\ML\Datasets\Unlabeled;
use Rubix\ML\Classifiers\GaussianNB;
use Rubix\ML\PersistentModel;
use Rubix\ML\Serializers\RBX;
use Rubix\ML\Persisters\Filesystem;
use Rubix\ML\CrossValidation\Metrics\FBeta;



In [1]:
$estimator = PersistentModel::load(new Filesystem('work/submit1.model'), new RBX());
echo "Loaded";


Loaded

In [2]:
$running_preds = [];

$handle = fopen("data/test_data.csv", "r");
$line = fgets($handle); // header
$fnames = explode(",", $line);
array_shift($fnames); // ID
$date = array_shift($fnames); // date, SP_2
array_unshift($fnames, $date . "_y", $date . "_m", $date . "_d");

$line = fgets($handle);

$BATCH_SIZE = 20000;

$total_samples = 0;
$samples = [];
$names   = [];
echo date(DATE_RFC2822) . '<br>';
while ($line !== false) {
  $fields = explode(",", $line);
  $id = array_shift($fields);
  $row=[];
  $date = array_shift($fields);
  [$y, $m, $d] = explode("-", $date);
  $row[] = floatval($y);
  $row[] = floatval($m);
  $row[] = floatval($d);
  foreach($fields as $v) {
      if($v === ""){
         $v = 0;
      }else{
         $v = floatval($v);
      }
      $row[] = $v;
  }
  $samples[] = $row;
  $names[]   = $id;

  if(count($samples) >= $BATCH_SIZE) {
     $testset = new Unlabeled($samples);
     $predictions = $estimator->predict($testset);

     for($idx=0; $idx<count($samples); $idx++){
       $id = $names[$idx];
       if(! isset($running_preds[$id])){
         $running_preds[$id] = [];
       }
       $running_preds[$id][] = $predictions[$idx];
     }
     $samples = [];
     $names   = [];
  }
  $line = fgets($handle);
}
if($samples) {
  $testset = new Unlabeled($samples);
  $predictions = $estimator->predict($testset);

  for($idx=0; $idx<count($samples); $idx++){
    $id = $names[$idx];
    if(! isset($running_preds[$id])){
      $running_preds[$id] = [];
    }
    $running_preds[$id][] = $predictions[$idx];
  }
}
fclose($handle);
echo 'Predicted<br>';
echo date(DATE_RFC2822) . '<br>';
$output=[ "customer_ID,prediction" ];
foreach($running_preds as $id => $preds) {
  $score = array_sum($preds) * 1.0 / count($preds);
  $output[] = "$id,$score";
}
file_put_contents("work/submission1.csv", implode("\n", $output)."\n");
echo 'Saved<br>';
echo date(DATE_RFC2822) . '<br>';


Sat, 11 Jun 2022 02:43:37 -0700 Predicted Sat, 11 Jun 2022 02:58:25 -0700 Saved Sat, 11 Jun 2022 02:58:26 -0700